### Code

In [1]:
%pip install -q "ipywidgets"
import ipywidgets as widgets

Note: you may need to restart the kernel to use updated packages.


In [2]:
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display
from scipy.optimize import root

In [3]:
def rootO(q,nO,qi):
    return nO*np.cos(q)-np.sin(qi)

In [4]:
def uN_E(t,t0,nO,nE):
    uu = np.array([nO*np.sin(t0)*np.sin(t)+nE*np.cos(t0)*np.cos(t),-nO*np.cos(t0)*np.sin(t)+nE*np.sin(t0)*np.cos(t)])
    return uu/np.linalg.norm(uu)

In [5]:
def uT_E(t,t0,nO,nE):
    uu = np.array([-nO*np.cos(t0)*np.sin(t)+nE*np.sin(t0)*np.cos(t),-nO*np.sin(t0)*np.sin(t)-nE*np.cos(t0)*np.cos(t)])
    return uu/np.linalg.norm(uu)

In [6]:
def uN_O(t):
    return np.array([np.cos(t),-np.sin(t)])

In [7]:
def uT_O(t):
    return np.array([-np.sin(t),np.cos(t)])

In [8]:
def update_plot(nE, nO, t0, qi):

    xmax = max(nE,nO)
    
    #rotation
    R = np.array([[np.cos(t0),-np.sin(t0)],[np.sin(t0),np.cos(t0)]])
    
    # straordinario
    if t0<=np.pi/2.:
        ti = np.arctan(nO/nE*np.tan(t0))
    else:
        ti = np.pi+np.arctan(nO/nE*np.tan(t0-np.pi))

    t = np.linspace(ti, np.pi+ti, 100)

    xy = np.array([nO*np.cos(t),-nE*np.sin(t)])
    #xyR = np.dot(R,xy)
    xyR=np.array([nO*np.cos(t0)*np.cos(t)+nE*np.sin(t0)*np.sin(t),
                 nO*np.sin(t0)*np.cos(t)-nE*np.cos(t0)*np.sin(t)])


    #ordinario
    tC = -np.linspace(0,np.pi,100)
    Ox = nO*np.cos(tC)
    Oy = nO*np.sin(tC)

    #isotropo
    tI = np.linspace(0,np.pi,100)
    xI = np.cos(tI)
    yI = np.sin(tI)

    #optical axis
    A = [0.1,0.]
    B = [-nO-0.1,0.]
    AR = np.dot(R,A)
    BR = np.dot(R,B)

    #incoming ki
    I = [-np.sin(qi),np.cos(qi)]

    #find O root
    #qO = root(rootO,np.pi/2-qi,args=(nO,qi)).x[0]
    qO=np.arccos(np.sin(qi)/nO)
    xO = nO*np.cos(qO)
    yO = -nO*np.sin(qO)

    #find E root
    qE=2*np.arctan((nE*np.sin(t0)+np.sqrt(nO**2*np.cos(t0)**2-np.sin(qi)**2+nE**2*np.sin(t0)**2))/(nO*np.cos(t0)+np.sin(qi)))
    xE = nO*np.cos(t0)*np.cos(qE)+nE*np.sin(t0)*np.sin(qE)
    yE = (nO*np.sin(t0)*np.cos(qE)-nE*np.cos(t0)*np.sin(qE))
    

    fig, ax = plt.subplots(figsize=(8,8))
    plt.gca().set_aspect('equal')
    ax.clear()
    ax.set_xticks([])
    ax.set_yticks([])

    shift_xmin = 1.
    ax.set_xlim(-xmax-0.2,xmax+0.2)
    ax.set_ylim(-xmax-shift_xmin,1.2)
    
    ax.plot(xyR[0], xyR[1], color='orange')
    ax.plot(Ox,Oy,c='b')
    ax.plot(xI,yI,c='k')
    ax.plot([AR[0],BR[0]],[AR[1],BR[1]],c='k',ls='--')


    # show angles
    if xE!=0:
        phEran = np.linspace(np.arctan(yE/xE),-np.pi/2.,100)
    else:
        phEran = np.linspace(np.pi/2.,-np.pi/2.,100)
    rE = .5
    if qi!=0:
        ax.plot(rE*np.cos(phEran),rE*np.sin(phEran),c='orange')
        ax.annotate(r'$\varphi_E$',(-.18,-rE-.05),c='orange',fontsize=12)

    phOran = np.linspace(np.arctan(yO/xO),-np.pi/2.,100)
    rO = .3
    ax.plot(rO*np.cos(phOran),rO*np.sin(phOran),c='blue')
    ax.annotate(r'$\varphi_O$',(-.18,-rO-.05),c='b',fontsize=12)

    phIran = np.linspace(np.pi/2,np.pi/2+qi,100)
    ax.plot(rO*np.cos(phIran),rO*np.sin(phIran),c='r')
    ax.annotate(r'$\varphi_i$',(.01,rO),c='r',fontsize=12)

    
    #plot incoming
    #x.plot([I[0],0],[I[1],0],c='r')
    ax.annotate('',[0,0],I,arrowprops=dict(facecolor='r', edgecolor='r', width=3, shrink=0.03))
    ax.annotate(r'$\vec{k}_i$',[I[0]/2.+.015,I[1]/2+.015],c='r',fontsize=12)

    #plot O outgoing
    ax.annotate('',[xO,yO],[0,0],arrowprops=dict(facecolor='b', edgecolor='b', width=3, shrink=0.03))
    ax.annotate(r'$\vec{k}_O$',[xO/2.5+.05,yO/2.5+.01],c='b',fontsize=12,zorder=1)

    #plot E outgoing
    #ax.plot([0,xE],[0,yE],c='orange')
    ax.annotate('',[xE,yE],[0,0],arrowprops=dict(facecolor='orange', edgecolor='orange', width=3, shrink=0.03),zorder=0)
    ax.annotate(r'$\vec{k}_E$',[xE/1.5+0.05,yE/1.5+.01],c='orange',fontsize=12)

    # S_E
    uE = uN_E(qE,t0,nO,nE)
    ax.annotate('',np.array([xE,yE])+uE,np.array([xE,yE]),arrowprops=dict(facecolor='orange', edgecolor='orange', width=3, shrink=0.03))
    ax.annotate(r'$\vec{S}_E$',np.array([xE,yE])+uE/1.5+np.array([0.02,0.02]),fontsize=12,c='orange')

    uTE = 0.5*uT_E(qE,t0,nO,nE)
    ax.plot([xE-uTE[0],xE+uTE[0]],[yE-uTE[1],yE+uTE[1]],c='m')

    # S_O
    uO = uN_O(qO)
    ax.annotate('',np.array([xO,yO])+uO,np.array([xO,yO]),arrowprops=dict(facecolor='b', edgecolor='b', width=3, shrink=0.03))
    ax.annotate(r'$\vec{S}_O$',np.array([xO,yO])+uO/2.5+np.array([0.02,0.02]),fontsize=12,c='b')

    uTO = 0.5*uT_O(qO)
    ax.plot([xO-uTO[0],xO+uTO[0]],[yO+uTO[1],yO-uTO[1]],c='m')

    
    

    if qi!=0:
        ax.annotate('',I,[0,I[1]], arrowprops=dict(arrowstyle='<->'))
        ax.annotate(r'$k_i\sin\theta_i$',[I[0]/2.,I[1]+0.04],ha='center')
    ax.axvline(I[0],c='k',ls='-.',lw=.5)
    ax.annotate('',[0,I[1]],[-I[0],I[1]], arrowprops=dict(arrowstyle='<->'))
    ax.axvline(-I[0],c='k',ls='-.',lw=.5)
    
    ax.scatter(0,0,marker='o',c='k')
    ax.axhline(0,c='k',lw=.5)
    ax.axvline(0,c='k',lw=.5)
    ax.axhspan(-xmax-shift_xmin,0,color='k',alpha=0.05)


    
    plt.show()

In [9]:
initial_nE = 1.5
initial_nO = 1.5
initial_t0 = 0.0
initial_qi = np.pi/4.

# Create sliders for adjusting slopes
nE_slider = widgets.FloatSlider(value=initial_nE, min=1, max=2, step=0.02, description=r'$n_E$ :')
nO_slider = widgets.FloatSlider(value=initial_nO, min=1, max=2, step=0.02, description='$n_O$ :')
t0_slider = widgets.FloatSlider(value=initial_t0, min=0, max=np.pi, step=0.01, description=r'$\theta_0$ (rad):')
qi_slider = widgets.FloatSlider(value=initial_qi, min=0, max=np.pi/2.-0.00, step=0.01, description=r'$\theta_i$ (rad):')

In [10]:
interactive_plot = widgets.interactive(update_plot, nE=nE_slider, nO=nO_slider, t0=t0_slider, qi=qi_slider)

# Birifrangenza

### Descrizione
Rifrazione di un raggio incidente da un mezzo isotropo in un mezzo uniassiale.</br>
È possibile variare:
- l'angolo di incidenza $\theta_i$,
- la direzione $\theta_0$ dell'**asse ottico** del materiale uniassiale,
- gli indici di rifrazione $n_E$ ed $n_O$ del mezzo uniassiale.

$\varphi_O$ e $\varphi_E$ indicano gli angoli di rifrazione del raggio ordinario e straordinario, rispettivamente. $\vec{S}_O$ ed $\vec{S}_E$ indicano i vettori di Poynting del raggio ordinario e straordinario.

### Utilizzo
Per eseguire il notebook, dal menu in alto: `Run > Run All Cells`

In [11]:
display(interactive_plot)

interactive(children=(FloatSlider(value=1.5, description='$n_E$ :', max=2.0, min=1.0, step=0.02), FloatSlider(…